In [1]:
import sys

import polars as pl
import torch
from data_loader.TimeSeriesModule import MultiPartDataModule
from models.Titans import TitanConfig, LMMModel
from model_runner.model_train import TitanTrain

'''
pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128
https://developer.nvidia.com/cuda-12-8-0-download-archive
'''

MAC_DIR = '../data/'
WINDOW_DIR = 'C:/Users/USER/PycharmProjects/research/data/'

if sys.platform == 'win32':
    DIR = WINDOW_DIR
    print(torch.cuda.is_available())
    print(torch.cuda.device_count())
    print(torch.version.cuda)
    print(torch.__version__)
    print(torch.cuda.get_device_name(0))
    print(torch.__version__)
else:
    DIR = MAC_DIR

True
1
12.8
2.9.0.dev20250716+cu128
NVIDIA GeForce RTX 5080
2.9.0.dev20250716+cu128


In [ ]:
target_dyn_demand = pl.read_parquet(DIR + 'target_dyn_demand.parquet')

data_module = MultiPartDataModule(
    df = target_dyn_demand,
    config = TitanConfig(),
    batch_size = 128,
    val_ratio = 0.2
)

train_loader = data_module.get_train_loader()
val_loader = data_module.get_val_loader()
inference_loader = data_module.get_inference_loader()

model = LMMModel()

train_with_tta = TitanTrain().train_model_with_tta(
    model = model,
    train_loader = train_loader,
    val_loader = val_loader
)


In [ ]:
torch.save(model.state_dict(), DIR + 'titan_tta_20250912_l12_h_60.pt')

In [ ]:
preds, trues = TitanTrain().evaluate(
    model = model,
    loader = inference_loader,
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
)